In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torch.nn.functional as f

In [3]:
# Load MNIST data
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Hyper parameters
learning_rate = 1e-2
batch_size = 100
num_classes = 10
num_epochs = 10

In [14]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=30, kernel_size=5)

        self.fc1 = nn.Linear(in_features=30 * 8 * 8, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = f.relu(self.conv2(x))
        x = self.pool(f.relu(self.conv3(x)))
        x = x.view(-1, 30 * 8 * 8)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}')
print('Done')

Epoch [1/10], Step [300/600], Loss: 2.3041
Epoch [1/10], Step [600/600], Loss: 1.5860
Epoch [2/10], Step [300/600], Loss: 0.3236
Epoch [2/10], Step [600/600], Loss: 0.3992
Epoch [3/10], Step [300/600], Loss: 0.3521
Epoch [3/10], Step [600/600], Loss: 0.2576
Epoch [4/10], Step [300/600], Loss: 0.0660
Epoch [4/10], Step [600/600], Loss: 0.1685
Epoch [5/10], Step [300/600], Loss: 0.1472
Epoch [5/10], Step [600/600], Loss: 0.0950
Epoch [6/10], Step [300/600], Loss: 0.0949
Epoch [6/10], Step [600/600], Loss: 0.1051
Epoch [7/10], Step [300/600], Loss: 0.1097
Epoch [7/10], Step [600/600], Loss: 0.2178
Epoch [8/10], Step [300/600], Loss: 0.1293
Epoch [8/10], Step [600/600], Loss: 0.1029
Epoch [9/10], Step [300/600], Loss: 0.0204
Epoch [9/10], Step [600/600], Loss: 0.0707
Epoch [10/10], Step [300/600], Loss: 0.0535
Epoch [10/10], Step [600/600], Loss: 0.0264
Done


In [16]:
with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in test_loader:

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total
    print(f'Accuracy: {100 * acc:.2f}%')

Accuracy: 97.41%
